In [1]:
from LOTlib3.Grammar import Grammar
from LOTlib3.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib3.Hypotheses.Likelihoods.BinaryLikelihood import BinaryLikelihood
from LOTlib3.DataAndObjects import FunctionData, Obj # for nicely managing data

from LOTlib3 import break_ctrlc
from LOTlib3.Miscellaneous import qq
from LOTlib3.TopN import TopN
from LOTlib3.Samplers.MetropolisHastings import MetropolisHastingsSampler

# Grammar

In [11]:
grammar = Grammar()

grammar.add_rule('START', '', ['QUANT'], 1.0)

# Very simple -- one allowed and required quantifier
# grammar.add_rule('QUANT', 'exists_', ['FUNCTION', 'SET'], 1.00)
grammar.add_rule('QUANT', 'forall_', ['FUNCTION', 'SET'], 1.00)

# The thing we are a function of
grammar.add_rule('SET', 'S', None, 1.0)

# And allow us to create a new kind of function
grammar.add_rule('FUNCTION', 'lambda', ['BOOL'], 1.0, bv_type='OBJECT')

# Logical connectives
grammar.add_rule('BOOL', 'and_', ['BOOL', 'BOOL'], 1.0)
grammar.add_rule('BOOL', 'or_', ['BOOL', 'BOOL'], 1.0)
# grammar.add_rule('BOOL', 'not_', ['BOOL'], 1.0)

# non-terminal arguments get passed as normal python arguments
grammar.add_rule('BOOL', 'is_color_',  ['OBJECT', '\'red\''],   5.00) # --> is_color_(OBJECT, 'red') --> OBJECT.color == 'red'
grammar.add_rule('BOOL', 'is_color_',  ['OBJECT', '\'blue\''],  5.00)
grammar.add_rule('BOOL', 'is_color_',  ['OBJECT', '\'green\''], 5.00)


BOOL -> is_color_['OBJECT', "'green'"]	w/ p=5.0

# Data

In [12]:
def make_data(n=1):
    return [ FunctionData(input=[ {Obj(color='red'), Obj(color='red'), Obj(color='green')} ], output=True, alpha=0.99) ]*n


data = make_data()
print(data)

[<{<OBJECT: color=green >, <OBJECT: color=red >, <OBJECT: color=red >} -> True>]


# Inference

In [13]:
class MyHypothesis(BinaryLikelihood, LOTHypothesis):
    def __init__(self, grammar=grammar, **kwargs):
        LOTHypothesis.__init__(self, grammar=grammar, display='lambda S: %s', **kwargs)


h   = MyHypothesis()
print('H: {}; prior = {:.4f}; lik = {:.4f}'.format(h, h.compute_prior(), h.compute_likelihood(data)))

if False:
    top  = TopN(N=10)
    thin = 100
    for i, h in enumerate(break_ctrlc(MetropolisHastingsSampler(h0, data))):
        top << h
        if i % thin == 0:
            print("#", i, h.posterior_score, h.prior, h.likelihood, qq(h))

    for h in top:
        print(h.posterior_score, h.prior, h.likelihood, qq(h))

NameError in function call:  name 'OBJECT' is not defined  ;  lambda S: is_color_(OBJECT, 'red') ({<OBJECT: color=green >, <OBJECT: color=red >, <OBJECT: color=red >},)


/opt/miniconda3/envs/iac/lib/python3.10/site-packages/LOTlib3/Miscellaneous.py:190: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and (slot is not "__doc__") and (slot is not "__module__"):
/opt/miniconda3/envs/iac/lib/python3.10/site-packages/LOTlib3/Miscellaneous.py:190: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and (slot is not "__doc__") and (slot is not "__module__"):
/opt/miniconda3/envs/iac/lib/python3.10/site-packages/LOTlib3/Miscellaneous.py:190: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and (slot is not "__doc__") and (slot is not "__module__"):
/opt/miniconda3/envs/iac/lib/python3.10/site-packages/LOTlib3/Miscellaneous.py:190: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and (slot is not "__doc__") and (slot is not "__module__"):


NameError: 